In [ ]:
!pip install arxiv

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [2]:
# api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
# wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

from langchain_core.tools import Tool
import wikipedia
wiki_tool = Tool (
    name="Wikepedia" ,
    func= lambda q: wikipedia.run(q),
    description="Useful for when you need to look up a topic or person on Wikipedia."
)


In [3]:
wiki_tool.name

'Wikepedia'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/docs/")

docs=loader.load()

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings=HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)
vectordb=FAISS.from_documents(documents,embedding=embeddings)
retriever=vectordb.as_retriever()
retriever


USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\vishn\AppData\Local\Temp\ipykernel_19676\1377896551.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024EE230C2F0>, search_kwargs={})

In [9]:
from langchain_core.tools import create_retriever_tool
retriever_tool=create_retriever_tool(
                        retriever=retriever,
                        name="langsmith_search",
                      description="Search for information about LangSmith,For any question about LangSmith, use this tool to find the answer from the documentation."
                      )


In [ ]:
retriever_tool.name

In [5]:
#Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool=Tool (
    name="arxiv_search",
    func= lambda q: arxiv_wrapper.run(q),
    description="Search Arxiv for academic papaers."
)
arxiv_tool.name

'arxiv_search'

In [10]:
tools=[wiki_tool,arxiv_tool,retriever_tool]

In [ ]:
tools

In [11]:
from langchain_ollama import ChatOllama

llm=ChatOllama(model="gemma2:2b")
llm

ChatOllama(model='gemma2:2b')

In [ ]:
### Deprecated way to get prompt



# from langchain_classic import hub
# #Get the prompt to use - you can modify this!

# prompt=hub.pull("hwchase17/openai-functions-agent")
# prompt.messages



##current way to get prompt

# from langchain_core.messages import SystemMessage

# prompt=SystemMessage(
#     content="You are a helpful AI agent that uses tools to answer user questions."
# )

In [12]:
# --- AGENT SETUP USING LCEL (new LangChain way) ---
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful AI agent that uses tools to answer user questions."
)

TypeError: functools.partial(<function _get_relevant_documents at 0x0000024EBB02D4E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024EE230C2F0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content') is not a module, class, method, or function.

In [ ]:
## Agent Execution

from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

agent_executor